#### 二、编辑距离（Minimum Edit Distance，MED）
- Levenshtein Distance 是用来度量两个序列相似程度的指标
- 将单词 w1 转换为单词 w2 所需要的最少单字符编辑操作次数。单字符编辑操作如下三种：
    - 插入
    - 删除
    - 替换
- 如 "kitten" 和 "sitting"，编辑距离为 3：
    - kitten → sitten（替换）
    - sitten → sittin（替换）
    - sittin → sitting（插入）

In [1]:
# 求解编辑距离
def edit_distance(str1, str2):
    n = len(str1)
    m = len(str2)

    dp = [[0] * (m + 1) for _ in range(n + 1)]

    for j in range(1, m + 1):
        dp[0][j] = dp[0][j - 1] + 1
    for i in range(1, n + 1):
        dp[i][0] = dp[i - 1][0] + 1

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = min(dp[i - 1][j], dp[i][j - 1],
                               dp[i - 1][j - 1]) + 1
    return dp[-1][-1]

In [1]:
# 求解编辑距离的同时，获取编辑路径
def edit_distance(str1, str2):
    matrix = [[i + j for j in range(len(str2) + 1)]
              for i in range(len(str1) + 1)]

    operation_matrix = [['' for j in range(len(str2) + 1)]
                        for i in range(len(str1) + 1)]
    for j in range(1, len(str2) + 1):
        operation_matrix[0][j] = operation_matrix[0][j - 1] + 'ADD {};'.format(
            str2[j - 1])
    for i in range(1, len(str1) + 1):
        operation_matrix[i][0] = operation_matrix[i - 1][0] + 'DEL {};'.format(
            str1[i - 1])

    for i in range(1, len(str1) + 1):
        for j in range(1, len(str2) + 1):
            if (str1[i - 1] == str2[j - 1]):
                d = 0
                operation = ''
            else:
                d = 1
                operation = 'SUB {}=>{};'.format(str1[i - 1], str2[j - 1])

            matrix[i][j], operation_matrix[i][j] = min(
                (matrix[i - 1][j - 1] + d,
                 operation_matrix[i - 1][j - 1] + operation),
                (matrix[i - 1][j] + 1,
                 operation_matrix[i - 1][j] + 'DEL {};'.format(str1[i - 1])),
                (matrix[i][j - 1] + 1,
                 operation_matrix[i][j - 1] + 'ADD {};'.format(str2[j - 1])))

    return matrix[-1][-1], operation_matrix[-1][-1]

In [2]:
edit_distance('jieba', 'beijing')

(6, 'ADD b;ADD e;ADD i;DEL e;SUB b=>n;SUB a=>g;')

In [1]:
# 记忆化求解
from functools import lru_cache
solution = {}


@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    if min(len(string1), len(string2)) == 0:
        return max(len(string1), len(string2))

    tail_s1 = string1[-1]
    tail_s2 = string2[-1]

    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2))
    ]

    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1,
                        'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)

    min_distance, operation = min(candidates, key=lambda x: x[0])
    solution[(string1, string2)] = operation

    return min_distance


edit_distance('ABCDE', 'ABCCEF'), solution

(2,
 {('A', 'A'): '',
  ('A', 'AB'): 'ADD B',
  ('A', 'ABC'): 'ADD C',
  ('A', 'ABCC'): 'ADD C',
  ('A', 'ABCCE'): 'ADD E',
  ('A', 'ABCCEF'): 'ADD F',
  ('AB', 'A'): 'DEL B',
  ('AB', 'AB'): '',
  ('AB', 'ABC'): 'ADD C',
  ('AB', 'ABCC'): 'ADD C',
  ('AB', 'ABCCE'): 'ADD E',
  ('AB', 'ABCCEF'): 'ADD F',
  ('ABC', 'A'): 'DEL C',
  ('ABC', 'AB'): 'DEL C',
  ('ABC', 'ABC'): '',
  ('ABC', 'ABCC'): 'ADD C',
  ('ABC', 'ABCCE'): 'ADD E',
  ('ABC', 'ABCCEF'): 'ADD F',
  ('ABCD', 'A'): 'DEL D',
  ('ABCD', 'AB'): 'DEL D',
  ('ABCD', 'ABC'): 'DEL D',
  ('ABCD', 'ABCC'): 'SUB D => C',
  ('ABCD', 'ABCCE'): 'ADD E',
  ('ABCD', 'ABCCEF'): 'ADD F',
  ('ABCDE', 'A'): 'DEL E',
  ('ABCDE', 'AB'): 'DEL E',
  ('ABCDE', 'ABC'): 'DEL E',
  ('ABCDE', 'ABCC'): 'DEL E',
  ('ABCDE', 'ABCCE'): '',
  ('ABCDE', 'ABCCEF'): 'ADD F'})

In [ ]:
# 自顶向下，递归
def Levenshtein_Distance_Recursive(str1, str2):

    if len(str1) == 0:
        return len(str2)
    elif len(str2) == 0:
        return len(str1)
    elif str1 == str2:
        return 0

    if str1[len(str1) - 1] == str2[len(str2) - 1]:
        d = 0
    else:
        d = 1

    return min(
        Levenshtein_Distance_Recursive(str1, str2[:-1]) + 1,
        Levenshtein_Distance_Recursive(str1[:-1], str2) + 1,
        Levenshtein_Distance_Recursive(str1[:-1], str2[:-1]) + d)

In [ ]:
# 自底向上，动态规划
def Levenshtein_Distance(str1, str2):
    """
    计算字符串 str1 和 str2 的编辑距离
    :param str1
    :param str2
    :return:
    """
    matrix = [[i + j for j in range(len(str2) + 1)]
              for i in range(len(str1) + 1)]

    for i in range(1, len(str1) + 1):
        for j in range(1, len(str2) + 1):
            if (str1[i - 1] == str2[j - 1]):
                d = 0
            else:
                d = 1

            matrix[i][j] = min(matrix[i - 1][j] + 1, matrix[i][j - 1] + 1,
                               matrix[i - 1][j - 1] + d)

    return matrix[-1][-1]